In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('deliverytime.csv')

In [3]:
df = df.loc[:, ['Delivery_person_Age', 'Delivery_person_Ratings','Type_of_order','Type_of_vehicle','Time_taken(min)']]

# Generate 'distance' using a proportional scaling approach
# Min-Max Scaling to Map Time to Distance 
df['distance'] = (df['Time_taken(min)'] - df['Time_taken(min)'].min()) / (df['Time_taken(min)'].max() - df['Time_taken(min)'].min()) * 30 + 1

# Round to integer values
df['distance'] = df['distance'].round().astype(int)

df.sample(5)

,Delivery_person_Age,Delivery_person_Ratings,Type_of_order,Type_of_vehicle,Time_taken(min),distance
44232,35,4.8,Drinks,motorcycle,28,13
38238,31,4.9,Drinks,scooter,42,23
3477,32,4.7,Meal,motorcycle,36,19
8764,22,4.9,Drinks,motorcycle,35,18
44197,22,4.8,Drinks,scooter,14,4


In [4]:
cat_col = columns=['Type_of_order', 'Type_of_vehicle']

trf1 = ColumnTransformer([
    ('ohe',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),cat_col)
],remainder='passthrough')

trf2 = LinearRegression()

pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2)
])

# Separate features and target
X = df.drop('Time_taken(min)', axis=1)
y = df['Time_taken(min)']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)

In [5]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R2 Score: {r2}')


MAE: 0.3549180485113136
MSE: 0.17015384286392793
R2 Score: 0.9980593355514586


In [6]:
# Save the trained model using joblib
joblib.dump(pipe, 'timelytics_model.pkl')
print("Model saved as 'timelytics_model.pkl'")


Model saved as 'timelytics_model.pkl'
